# Transfer matrix approach to unitary optimizatioin - testing
Created 30/05/2024

Objectives:
* Test the transfer_matrix_approach_to_unitary_optimization.ipynb notebook.

# Import packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import h5py
from tenpy.tools import hdf5_io
import tenpy
import tenpy.linalg.np_conserved as npc

import os

In [3]:
from collections import Counter
from functools import reduce

In [4]:
from spt_classification import (
    get_transfer_matrix_from_unitary,
    get_transfer_matrices_from_unitary_list,
    multiply_transfer_matrices,
    to_npc_array
)

# Load data

In [6]:
DATA_DIR = r"data/transverse_cluster_200_site_dmrg"

In [7]:
loaded_data = list()

for local_file_name in os.listdir(DATA_DIR):
    f_name = r"{}/{}".format(DATA_DIR, local_file_name, ignore_unknown=False)
    with h5py.File(f_name, 'r') as f:
        data = hdf5_io.load_from_hdf5(f)
        loaded_data.append(data)

In [8]:
b_parameters = sorted(list(d['paramters']['B'] for d in loaded_data))

In [9]:
psi_dict = dict()

In [10]:
for b in b_parameters:
    psi = next(
        d['wavefunction']
        for d in loaded_data
        if d['paramters']['B'] == b
    )

    rounded_b = round(b, 1)
    psi_dict[rounded_b] = psi

In [11]:
list(psi_dict)

[0.0,
 0.1,
 0.2,
 0.3,
 0.4,
 0.5,
 0.6,
 0.7,
 0.8,
 0.9,
 1.0,
 1.1,
 1.2,
 1.3,
 1.4,
 1.5,
 1.6,
 1.7,
 1.8,
 1.9,
 2.0]

In [12]:
test_psi = psi_dict[0.5]

# Code

In [13]:
np_I = np.array([[1,0],[0,1]])
np_X = np.array([[0,1],[1,0]])
np_Y = np.array([[0,-1j],[1j,0]])
np_Z = np.array([[1,0],[0,-1]])

In [14]:
import math
C1 = math.sqrt(2)
C2 = 1.533751168755204288118041

In [15]:
def super_fibonacci_point(n, s, phi=C1, psi=C2):
    t = s/n
    d = 2*math.pi*s
    r = math.sqrt(t)
    R = math.sqrt(1-t)
    alpha = d/phi
    beta = d/psi
    point = [
        r*math.sin(alpha),
        r*math.cos(alpha),
        R*math.sin(beta),
        R*math.cos(beta)
    ]

    return point

def super_fibonacci(n, phi=C1, psi=C2):
    out = np.zeros((n,4))

    for i in range(n):
        s = i + 0.5
        point = super_fibonacci_point(n, s, phi=C1, psi=C2)
        out[i] = point

    return out

In [16]:
def s3_to_unitary(p):
    X = p[0]*np_I + 1j*(p[1]*np_X + p[2]*np_Y + p[3]*np_Z)
    return X

In [17]:
def operator_norm(m):
    singular_values = np.linalg.svd(m).S
    singular_values_counts = Counter(np.round(singular_values, 3))

    norm = max(singular_values_counts.keys())
    count = singular_values_counts[norm]

    return (norm, count)

In [20]:
def get_left_environment(psi, index):
    left_leg = psi.get_B(index).legs[0]
    SL = npc.diag(psi.get_SL(index), left_leg, labels = ['vL', 'vR'])
    left_environment = (
        npc.tensordot(SL, SL.conj(), (['vL',], ['vL*',]))
        .combine_legs([['vR', 'vR*'],])
        .to_ndarray()
    )

    return left_environment

In [18]:
def search_step(previous_points, s3_previous_points, s3_sample_points, num_next_points, index, vR):
    base_tms = np.array([
        get_transfer_matrix_from_unitary(test_psi, u, index)
        .combine_legs([['vL', 'vL*'], ['vR', 'vR*']])
        .to_ndarray()
        for u in base_unitaries
    ])

    base_vectors = np.matmul(previous_points, base_tms)
    base_overlaps = np.dot(base_vectors, vR)

    overlaps = np.abs(np.tensordot(s3_sample_points, base_overlaps, [[1,], [0,]]))
    target_percentile = 100.0*(1.0 - min(1, num_next_points/(overlaps.size)))
    overlap_threshold = np.percentile(overlaps, target_percentile)

    all_next_points = np.tensordot(s3_sample_points, base_vectors, [[1,], [0,]])

    overlaps_filter = (overlaps > overlap_threshold)
    

    if s3_previous_points is None:
        assert previous_points.shape[0] == 1
        all_next_s3_points = s3_sample_points[:, np.newaxis, np.newaxis, :]
        next_s3_depth = 1
    else:
        prev_num_s3_points, prev_s3_depth, *_ = s3_previous_points.shape
        next_s3_depth = prev_s3_depth + 1

        num_s3_sample_points = s3_sample_points.shape[0]
        all_next_s3_points = np.zeros((num_s3_sample_points, prev_num_s3_points, next_s3_depth, 4))
        
        all_next_s3_points[:, :, :-1, :] = s3_previous_points[np.newaxis, ...]
        all_next_s3_points[:, :, -1, :] = s3_sample_points[:, np.newaxis, :]
    
    filtered_next_s3_points = all_next_s3_points[overlaps_filter]
    filtered_next_points = all_next_points[overlaps_filter]
    filtered_overlaps = overlaps[overlaps_filter]

    return (filtered_next_points, filtered_next_s3_points, filtered_overlaps)

In [19]:
def search_step_left(previous_points, s3_previous_points, s3_sample_points, num_next_points, index, vL):
    base_tms = np.array([
        get_transfer_matrix_from_unitary(test_psi, u, index)
        .combine_legs([['vL', 'vL*'], ['vR', 'vR*']])
        .to_ndarray()
        for u in base_unitaries
    ])

    base_vectors = np.tensordot(previous_points, base_tms, [[-1,], [2,]])
    base_overlaps = np.dot(base_vectors, vL)

    overlaps = np.abs(np.tensordot(s3_sample_points, base_overlaps, [[1,], [1,]]))
    target_percentile = 100.0*(1.0 - min(1, num_next_points/(overlaps.size)))
    overlap_threshold = np.percentile(overlaps, target_percentile)

    all_next_points = np.tensordot(s3_sample_points, base_vectors, [[1,], [1,]])

    overlaps_filter = (overlaps > overlap_threshold)

    if s3_previous_points is None:
        assert previous_points.shape[0] == 1
        all_next_s3_points = s3_sample_points[:, np.newaxis, np.newaxis, :]
        next_s3_depth = 1
    else:
        prev_num_s3_points, prev_s3_depth, *_ = s3_previous_points.shape
        next_s3_depth = prev_s3_depth + 1

        num_s3_sample_points = s3_sample_points.shape[0]
        all_next_s3_points = np.zeros((num_s3_sample_points, prev_num_s3_points, next_s3_depth, 4))
        
        all_next_s3_points[:, :, :-1, :] = s3_previous_points[np.newaxis, ...]
        all_next_s3_points[:, :, -1, :] = s3_sample_points[:, np.newaxis, :]

    filtered_next_s3_points = np.reshape(all_next_s3_points[overlaps_filter], (-1, next_s3_depth, 4))
    filtered_next_points = np.reshape(all_next_points[overlaps_filter], (-1, 64))
    filtered_overlaps = overlaps[overlaps_filter].flatten()
        
    return (filtered_next_points, filtered_next_s3_points, filtered_overlaps)

In [41]:
def s3_points_to_unitary(s3_points):
    return np.tensordot(s3_points, base_unitaries, [[-1,], [0,]])

# Testing

In [23]:
symmetry_tm = (
    reduce(
        multiply_transfer_matrices,
        get_transfer_matrices_from_unitary_list(
            test_psi,
            [np_X, np_I]*30,
            (test_psi.L)//2 -30
        )
    )
    .combine_legs([['vL', 'vL*'], ['vR', 'vR*']])
    .to_ndarray()
)

In [24]:
U, S, Vh = np.linalg.svd(symmetry_tm)

In [25]:
left_projected_symmetry_state = U[:,0]
right_projected_symmetry_state = Vh[0]

In [26]:
super_fib_points = super_fibonacci(1000)

In [31]:
vR = np.identity(8).reshape((64,))

In [33]:
base_unitaries = np.array([np_I, 1j*np_X, 1j*np_Y, 1j*np_Z])

## One site
### From search

In [28]:
N=1000

In [34]:
r_points_1, r_s3_points_1, r_overlaps_1 = search_step(
    right_projected_symmetry_state[np.newaxis, :],
    None,
    super_fib_points,
    N,
    (test_psi.L//2)+30,
    vR
)

In [35]:
np.max(r_overlaps_1)

7.897631266251278e-11

In [77]:
r_arg_1 = np.argmax(r_overlaps_1)

In [38]:
i = (test_psi.L//2)-30-1

l_points_1, l_s3_points_1, l_overlaps_1 = search_step_left(
    left_projected_symmetry_state[np.newaxis, :],
    None,
    super_fib_points,
    N,
    i,
    get_left_environment(test_psi, i)
)

In [39]:
np.max(l_overlaps_1)

0.33907274121853925

In [43]:
l_arg_1 = np.argmax(l_overlaps_1)

In [40]:
S[0]*np.max(r_overlaps_1)*np.max(l_overlaps_1)

5.352301545120342e-11

### Check results
#### Via expecation

In [45]:
r_unitary_1 = s3_points_to_unitary(r_s3_points_1[r_arg_1, 0])
l_unitary_1 = s3_points_to_unitary(l_s3_points_1[l_arg_1, 0])

In [46]:
np_operators = [
    l_unitary_1,
    *([np_X, np_I]*30),
    r_unitary_1
]

In [47]:
len(np_operators)

62

In [48]:
operators = [to_npc_array(X) for X in np_operators]

In [49]:
expectation = test_psi.expectation_value_multi_sites(operators, (test_psi.L//2)-30-1)

In [50]:
np.abs(expectation)

5.3501256950701365e-11

Very close agreement.
#### Via transfer matrices

In [52]:
transfer_matrices = get_transfer_matrices_from_unitary_list(
    test_psi,
    np_operators,
    (test_psi.L//2)-30-1
)

In [53]:
overall_tm = reduce(multiply_transfer_matrices, transfer_matrices)

In [54]:
np_overall_tm = (
    overall_tm
    .combine_legs([['vL', 'vL*'], ['vR', 'vR*']])
    .to_ndarray()
)

In [55]:
np.abs(reduce(np.dot, [get_left_environment(test_psi, (test_psi.L//2)-30-1), np_overall_tm, vR]))

5.350140431489429e-11

#### SVD product approximation

In [108]:
right_tm = (
    get_transfer_matrix_from_unitary(
        test_psi,
        r_unitary_1,
        (test_psi.L//2)+30
    )
    .combine_legs([['vL', 'vL*'], ['vR', 'vR*']])
    .to_ndarray()
)

In [60]:
left_tm = (
    get_transfer_matrix_from_unitary(
        test_psi,
        l_unitary_1,
        (test_psi.L//2) -30 -1
    )
    .combine_legs([['vL', 'vL*'], ['vR', 'vR*']])
    .to_ndarray()
)

In [70]:
right_term = np.abs(reduce(np.dot, [right_projected_symmetry_state, right_tm, vR]))

In [107]:
right_term

1.349953572573684

In [65]:
np.linalg.norm(np.dot(right_projected_symmetry_state, right_tm) - r_points_1[r_arg_1])

0.7433005401241385

In [63]:
left_term = np.abs(
    reduce(
        np.dot,
        [
            get_left_environment(test_psi, (test_psi.L//2)-30-1),
            left_tm,
            left_projected_symmetry_state
        ]
    )
)

In [64]:
left_term

0.3390727412185392

In [72]:
S[0]*left_term*right_term

5.35231719316077e-11

Lesson learned, be _very_ careful with indexing...! Should probably automate somehow.

## Two sites
### From search

In [109]:
r_points_2, r_s3_points_2, r_overlaps_2 = search_step(
    r_points_1,
    r_s3_points_1,
    super_fib_points,
    N,
    (test_psi.L//2)+30+1,
    vR
)

In [110]:
np.max(r_overlaps_2)

1.3499535725736844

In [80]:
r_arg_2 = np.argmax(r_overlaps_2)

In [81]:
i = (test_psi.L//2)-30-2

l_points_2, l_s3_points_2, l_overlaps_2 = search_step_left(
    l_points_1,
    l_s3_points_1,
    super_fib_points,
    N,
    i,
    get_left_environment(test_psi, i)
)

In [82]:
np.max(l_overlaps_2)

0.3389229141160786

In [83]:
l_arg_2 = np.argmax(l_overlaps_2)

In [111]:
S[0]*np.max(r_overlaps_2)*np.max(l_overlaps_2)

0.9144724104850446

### Check results
#### Via expecation

In [86]:
r_unitary_2_1 = s3_points_to_unitary(r_s3_points_2[r_arg_2, 0])
r_unitary_2_2 = s3_points_to_unitary(r_s3_points_2[r_arg_2, 1])
l_unitary_2_1 = s3_points_to_unitary(l_s3_points_2[l_arg_2, 0])
l_unitary_2_2 = s3_points_to_unitary(l_s3_points_2[l_arg_2, 1])

In [87]:
np_operators = [
    l_unitary_2_2,
    l_unitary_2_1,
    *([np_X, np_I]*30),
    r_unitary_2_1,
    r_unitary_2_2
]

In [88]:
len(np_operators)

64

In [89]:
operators = [to_npc_array(X) for X in np_operators]

In [90]:
expectation = test_psi.expectation_value_multi_sites(operators, (test_psi.L//2)-30-2)

In [91]:
np.abs(expectation)

0.9144724104850276


#### Via transfer matrices

In [92]:
transfer_matrices = get_transfer_matrices_from_unitary_list(
    test_psi,
    np_operators,
    (test_psi.L//2)-30-2
)

In [93]:
overall_tm = reduce(multiply_transfer_matrices, transfer_matrices)

In [94]:
np_overall_tm = (
    overall_tm
    .combine_legs([['vL', 'vL*'], ['vR', 'vR*']])
    .to_ndarray()
)

In [95]:
np.abs(reduce(np.dot, [get_left_environment(test_psi, (test_psi.L//2)-30-2), np_overall_tm, vR]))

0.9144724104850274

#### SVD product approximation

In [96]:
right_tm_2_1 = (
    get_transfer_matrix_from_unitary(
        test_psi,
        r_unitary_2_1,
        (test_psi.L//2)+30
    )
    .combine_legs([['vL', 'vL*'], ['vR', 'vR*']])
    .to_ndarray()
)

right_tm_2_2 = (
    get_transfer_matrix_from_unitary(
        test_psi,
        r_unitary_2_2,
        (test_psi.L//2)+31
    )
    .combine_legs([['vL', 'vL*'], ['vR', 'vR*']])
    .to_ndarray()
)

In [97]:
left_tm_2_1 = (
    get_transfer_matrix_from_unitary(
        test_psi,
        l_unitary_2_1,
        (test_psi.L//2) -30 -1
    )
    .combine_legs([['vL', 'vL*'], ['vR', 'vR*']])
    .to_ndarray()
)

left_tm_2_2 = (
    get_transfer_matrix_from_unitary(
        test_psi,
        l_unitary_2_2,
        (test_psi.L//2) -30 -2
    )
    .combine_legs([['vL', 'vL*'], ['vR', 'vR*']])
    .to_ndarray()
)

In [98]:
right_term = np.abs(reduce(np.dot, [right_projected_symmetry_state, right_tm_2_1, right_tm_2_2, vR]))

In [99]:
right_term

1.349953572573684

In [65]:
np.linalg.norm(np.dot(right_projected_symmetry_state, right_tm) - r_points_1[r_arg_1])

0.7433005401241385

In [100]:
left_term = np.abs(
    reduce(
        np.dot,
        [
            get_left_environment(test_psi, (test_psi.L//2)-30-2),
            left_tm_2_2,
            left_tm_2_1,
            left_projected_symmetry_state
        ]
    )
)

In [101]:
left_term

0.33892291411607856

In [102]:
S[0]*left_term*right_term

0.9144724104850441

## Three sites
### From search

In [112]:
r_points_3, r_s3_points_3, r_overlaps_3 = search_step(
    r_points_2,
    r_s3_points_2,
    super_fib_points,
    N,
    (test_psi.L//2)+30+2,
    vR
)

In [113]:
np.max(r_overlaps_3)

1.3493004237318182

In [114]:
r_arg_3 = np.argmax(r_overlaps_3)

In [115]:
i = (test_psi.L//2)-30-3

l_points_3, l_s3_points_3, l_overlaps_3 = search_step_left(
    l_points_2,
    l_s3_points_2,
    super_fib_points,
    N,
    i,
    get_left_environment(test_psi, i)
)

In [116]:
np.max(l_overlaps_3)

0.33869682256904254

In [117]:
l_arg_3 = np.argmax(l_overlaps_3)

In [118]:
S[0]*np.max(r_overlaps_3)*np.max(l_overlaps_3)

0.9134202219625596

### Check results
#### Via expecation

In [119]:
r_unitary_3_1 = s3_points_to_unitary(r_s3_points_3[r_arg_3, 0])
r_unitary_3_2 = s3_points_to_unitary(r_s3_points_3[r_arg_3, 1])
r_unitary_3_3 = s3_points_to_unitary(r_s3_points_3[r_arg_3, 2])
l_unitary_3_1 = s3_points_to_unitary(l_s3_points_3[l_arg_3, 0])
l_unitary_3_2 = s3_points_to_unitary(l_s3_points_3[l_arg_3, 1])
l_unitary_3_3 = s3_points_to_unitary(l_s3_points_3[l_arg_3, 2])

In [120]:
np_operators = [
    l_unitary_3_3,
    l_unitary_3_2,
    l_unitary_3_1,
    *([np_X, np_I]*30),
    r_unitary_3_1,
    r_unitary_3_2,
    r_unitary_3_3
]

In [121]:
len(np_operators)

66

In [122]:
operators = [to_npc_array(X) for X in np_operators]

In [123]:
expectation = test_psi.expectation_value_multi_sites(operators, (test_psi.L//2)-30-3)

In [124]:
np.abs(expectation)

0.913420221962542


#### Via transfer matrices

In [92]:
transfer_matrices = get_transfer_matrices_from_unitary_list(
    test_psi,
    np_operators,
    (test_psi.L//2)-30-2
)

In [93]:
overall_tm = reduce(multiply_transfer_matrices, transfer_matrices)

In [94]:
np_overall_tm = (
    overall_tm
    .combine_legs([['vL', 'vL*'], ['vR', 'vR*']])
    .to_ndarray()
)

In [95]:
np.abs(reduce(np.dot, [get_left_environment(test_psi, (test_psi.L//2)-30-2), np_overall_tm, vR]))

0.9144724104850274

#### SVD product approximation

In [96]:
right_tm_2_1 = (
    get_transfer_matrix_from_unitary(
        test_psi,
        r_unitary_2_1,
        (test_psi.L//2)+30
    )
    .combine_legs([['vL', 'vL*'], ['vR', 'vR*']])
    .to_ndarray()
)

right_tm_2_2 = (
    get_transfer_matrix_from_unitary(
        test_psi,
        r_unitary_2_2,
        (test_psi.L//2)+31
    )
    .combine_legs([['vL', 'vL*'], ['vR', 'vR*']])
    .to_ndarray()
)

In [97]:
left_tm_2_1 = (
    get_transfer_matrix_from_unitary(
        test_psi,
        l_unitary_2_1,
        (test_psi.L//2) -30 -1
    )
    .combine_legs([['vL', 'vL*'], ['vR', 'vR*']])
    .to_ndarray()
)

left_tm_2_2 = (
    get_transfer_matrix_from_unitary(
        test_psi,
        l_unitary_2_2,
        (test_psi.L//2) -30 -2
    )
    .combine_legs([['vL', 'vL*'], ['vR', 'vR*']])
    .to_ndarray()
)

In [98]:
right_term = np.abs(reduce(np.dot, [right_projected_symmetry_state, right_tm_2_1, right_tm_2_2, vR]))

In [99]:
right_term

1.349953572573684

In [65]:
np.linalg.norm(np.dot(right_projected_symmetry_state, right_tm) - r_points_1[r_arg_1])

0.7433005401241385

In [100]:
left_term = np.abs(
    reduce(
        np.dot,
        [
            get_left_environment(test_psi, (test_psi.L//2)-30-2),
            left_tm_2_2,
            left_tm_2_1,
            left_projected_symmetry_state
        ]
    )
)

In [101]:
left_term

0.33892291411607856

In [102]:
S[0]*left_term*right_term

0.9144724104850441